###  ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Lab 3.04 | Regression Metrics Lab

Exercise 1: Build a function called `r2_adj()` that will calculate $R^2_{adj}$ for a model. For inputs, you'll need `y_true`, `y_preds`, the average value of $y$, the number of variables $p$, and the number of observations $n$.

In [8]:
import sklearn.metrics as metrics
import statsmodels.api as sm
import numpy as np
import pandas as pd

/home/greg/miniconda3/envs/ga/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [5]:
ff = pd.read_csv('forestfires.csv')
ff.head(2)

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0


#### r2_adj_brems: (this does not agree with OLS.model.summary())

$$R^2_{adj} = 1 - \frac{\frac{1}{n-p-1}\sum_{i=1}^n(y_i - \hat{y}_i)^2}{\frac{1}{n-1}\sum_{i=1}^n(y_i - \bar{y})^2}$$

#### r2_adj_greg: (this appears to agree with OLS.model.summary())

$$R^2_{adj} = 1 - \frac{{(n-1)}{(1-\frac{\sum_{i=1}^n(\hat{y}_i - \bar{y})^2}{\sum_{i=1}^n(y_i - \bar{y})^2} )}}{(n-p-1)}$$

In [6]:
def r2_adj_brems(y_true, y_preds, p, n):
    y_mean = np.mean(y_true)
    numerator = np.sum(np.square(y_true - y_preds)) / (n - p - 1)
    denominator = np.sum(np.square(y_true - y_mean)) / (n - 1)
    return numerator / denominator

In [63]:
def r2_adj_greg(y_true, y_preds, p):
    n = len(y_preds)        #number of observations
    y_avg = np.mean(y_true) #average of true y-values
    
    num_sum = 0
    denom_sum = 0
    
    num = 1/(n-1)
    denom = 1/(n-p-1)
    
    for yp in (y_preds): #calculates sigma (sum) for numerator
        num_sum += (yp - y_avg)**2
    for yt in y_true:                  #calculates sigma (sum) for denomenator
        denom_sum += (yt - y_avg)**2
        
    return 1 - (((n - 1)*(1-(num_sum/denom_sum)))/(n-p-1)) #return adjusted r^2 value

Exercise 2: Using the forest fire data from the lesson, build two models in `statsmodels`, one of which should include at least one newly-engineered feature. 

Test your output of Exercise 1 by running `r2_adj()` on both models and by checking `.summary()` of the models. (Adjusted $R^2$ is in the summary.)

In [64]:
X = ff.drop(columns=['area', 'month', 'day'])
y = ff['area']

In [65]:
# add in a column of 1 for a y-intercept term
X = sm.add_constant(X)
# instantiate the model
mod = sm.OLS(y, X)
# fit the OLS regression to the data
mod = mod.fit()
# print out R^2 and R^2_adj
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   area   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.119
Date:                Wed, 31 Jan 2018   Prob (F-statistic):              0.345
Time:                        10:09:51   Log-Likelihood:                -2874.8
No. Observations:                 517   AIC:                             5772.
Df Residuals:                     506   BIC:                             5818.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.3693     63.019     -0.101      0.920    -130.181     117.443
X              1.9079      1.448      1.317      0.188      -0.938       4.753
Y              0.5692      2.736      0.208      0.835      -4.807       5.945
FFMC          -0.0392      0.661     -0.059      0.953      -1.337       1.259
DMC            0.0773      0.067      1.151      0.250      -0.055       0.209
DC            -0.0033      0.016     -0.200      0.841      -0.036       0.029
ISI           -0.7137      0.772     -0.925      0.355      -2.229       0.802
temp           0.8002      0.787      1.017      0.310      -0.746       2.347
RH            -0.2306      0.237     -0.972      0.332      -0.697       0.236
wind           1.5574      1.670      0.933      0.352      -1.724       4.839
rain          -3.4040      9.681     -0.352      0.725     -22.424      15.616
==============================================================================
Omnibus:                      975.065   Durbin-Watson:                   1.647
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           781330.782
Skew:                          12.570   Prob(JB):                         0.00
Kurtosis:                     191.782   Cond. No.                     1.40e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [66]:
# this is the result you'd get if you copied the formula or R^2_adj from 3.04-regression-metrics.ipynb 
r2_adj_brems(y, mod.predict(X), len(X.columns), len(X))

0.99967098403713284

In [68]:
# this is the result you get if you use the corrected formula above, from r2_adj_greg
# note that you MUST subtract 1 from p if you're using an intercept (constant) term. 
# note that mod.summary() automatically subtracts this from their summary results
# if you add the term with sm.add_constant()
r2_adj_greg(y, mod.predict(X), len(X.columns)-1)  # subtract 1 if you're using the constant

0.0023046503186719969

Exercise 3: Suppose we build two models. Model 1 has predicted outputs `y_preds_1` and Model 2 has predicted outputs `y_preds_2`. We now have the question we'll always have:

*(Okay, maybe which model should I pick?)*

Build a function:
- called `model_picker()`
- that accepts `y_true`, `y_preds_1`, and `y_preds_2` as inputs, and
- returns:
    - $R^2$, MSE, RMSE, MedAE, and MSLE for model 1,
    - $R^2$, MSE, RMSE, MedAE, and MSLE for model 2,
    - "Model 1 is the model to pick!" if all of Model 1's scores are better than Model 2,
    - "Model 2 is the model to pick!" if all of Model 2's scores are better than Model 1, or
    - "Neither model wins." if neither model is uniformly better than the other.


Exercise 4: Using the forest fire data from the lesson, build two models in `scikit-learn`, one of which should include at least one newly-engineered feature. 

Test your output of Exercise 1 by running `r2_adj()` on both models and by checking `.summary()` of the models. (Adjusted $R^2$ is in the summary.)

BONUS: Adapt `model_picker()` to include $R^2_{adj}$ in its analysis. Note that you'll have to add certain inputs that you used in `r2_adj()`.